Test action space sample

In [1]:
from sb3_contrib import MaskablePPO
import edge_relocation as er
# create environment
erEnv = er.EdgeRelEnv("topoconfig.json")
erEnv.action_space.sample()

13

Test masks

In [2]:
erEnv.reset()
print("App reqlatency: ", erEnv.mecApp.app_req_latency, "current MEC ID: ", erEnv.mecApp.current_MEC.id, "current cell ID: ", erEnv.mecApp.user_position)
for mec in erEnv.mec_nodes:
    print(mec.id, "cpu util: ", mec.cpu_utilization, "mem util: ", mec.memory_utilization)
print(erEnv.action_masks())

#please note that masks refers to the valid action of selecting 'mec1', mec2, mec3 etc, while the list below lists mecs in wrong order

App reqlatency:  15 current MEC ID:  mec7 current cell ID:  5
mec8 cpu util:  43 mem util:  43
mec9 cpu util:  50 mem util:  50
mec10 cpu util:  74 mem util:  74
mec7 cpu util:  27 mem util:  32
mec12 cpu util:  41 mem util:  41
mec13 cpu util:  32 mem util:  32
mec14 cpu util:  64 mem util:  64
mec11 cpu util:  79 mem util:  79
mec16 cpu util:  33 mem util:  33
mec17 cpu util:  35 mem util:  35
mec18 cpu util:  52 mem util:  52
mec15 cpu util:  60 mem util:  60
mec20 cpu util:  46 mem util:  46
mec21 cpu util:  23 mem util:  23
mec22 cpu util:  43 mem util:  43
mec19 cpu util:  19 mem util:  19
mec2 cpu util:  21 mem util:  21
mec3 cpu util:  26 mem util:  26
mec4 cpu util:  68 mem util:  68
mec5 cpu util:  71 mem util:  71
mec6 cpu util:  64 mem util:  64
mec1 cpu util:  24 mem util:  24
[False, True, True, True, False, False, True, True, True, True, False, True, True, True, False, False, False, False, False, False, False, False]


Test Maskable PPO

In [3]:
import edge_relocation as er

import datetime


# create environment
erEnv = er.EdgeRelEnv("topoconfig.json")
obs = erEnv.reset()
log_dir = "../logs/PPO/" + datetime.datetime.now().strftime("MultiEnVRewNormDelay%Y%m%d-%H%M%S")
model = MaskablePPO("MultiInputPolicy", erEnv, verbose=1, tensorboard_log = log_dir)
model.learn(total_timesteps=1000)

#Seems that for today, MaskablePPO is checking about the type of Space, and it accepts only spaces of gym, not yet a spaces of gymnasium. Further checks required

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AssertionError: Unsupported structured space '<class 'gymnasium.spaces.dict.Dict'>'